In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the data
INCIDENTS = '../../dataset/data-raw/incidents.csv'
POVERTYYEAY = '../../dataset/data-raw/povertyByStateYear.csv'
STATEDISHOUSE = '../../dataset/data-raw/year_state_district_house.csv'

incidents = pd.read_csv(INCIDENTS)
poverty = pd.read_csv(POVERTYYEAY)
state_district_house = pd.read_csv(STATEDISHOUSE)

incidents['state'] = incidents['state'].str.upper()
poverty['state'] = poverty['state'].str.upper()
state_district_house['state'] = state_district_house['state'].str.upper()

/tmp/ipykernel_4270/4112513518.py:10: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents = pd.read_csv(INCIDENTS)


## First look at the data
Show an aggregate description of each column 

In [ ]:
a = incidents.apply(lambda x: x.apply(lambda y: y if type(y) == str else np.nan))

In [115]:
# change all dtypes to object
a = a.astype(str)
a.describe()

,date,state,city_or_county,address,latitude,longitude,congressional_district,state_house_district,state_senate_district,participant_age1,...,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,n_participants,notes,incident_characteristics1,incident_characteristics2
count,239677,239677,239677,239677,239677,239677,239677,239677,239677,239677,...,239677,239677,239677,239677,239677,239677,239677,239677,239677,239677
unique,2437,51,12898,198038,1,1,1,1,1,1,...,1,1,1,1,1,1,1,136653,53,91
top,2017-01-01,ILLINOIS,Chicago,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,Shot - Wounded/Injured,nan
freq,342,17556,10814,16497,239677,239677,239677,239677,239677,239677,...,239677,239677,239677,239677,239677,239677,239677,81017,93926,97746


In [116]:
from tabulate import tabulate
for name, df in [('incidents', incidents), ('poverty', poverty), ('state_district_house', state_district_house)]:
    df = df.copy()
    print(f"----- {name} summary -----")
    print(f"records: {df.shape[0]}")
    colsumpd = pd.DataFrame(index=df.columns)
    colsumpd['not_null'] = df.notnull().sum()
    colsumpd['unique'] = df.nunique()
    colsumpd['null_count'] = df.isnull().sum()
    colsumpd['null_perc'] = (colsumpd['null_count'] / df.shape[0] * 100).round(2)
    colsumpd['type'] = (df.dtypes.astype(str)).str.upper()
    numeric_df = df.apply(pd.to_numeric, errors='coerce').describe()
    numeric_df.index = ['numeric_' + str(row) for row in numeric_df.index]
    not_num_df = df.apply(lambda x: x.apply(lambda y: y if type(y) == str else np.nan)).astype(str).describe()
    not_num_df.index = ['not_num_' + str(row) for row in not_num_df.index]
    # round all values of numeric_df to 4 decimal places
    numeric_df = numeric_df.round(4)
    # merge the two dataframes on common columns
    colsumpd = pd.merge(colsumpd, numeric_df.T, left_index=True, right_index=True, how='outer')
    colsumpd = pd.merge(colsumpd, not_num_df.T, left_index=True, right_index=True, how='outer')
    print(tabulate(colsumpd.T, headers='keys', tablefmt='pipe', numalign='left', stralign='left'))
    # for col in df.columns:
    #     print(f"{col}: {df[col].isnull().sum()}")


----- incidents summary -----
records: 239677
|                | date       | state    | city_or_county   | address   | latitude   | longitude   | congressional_district   | state_house_district   | state_senate_district   | participant_age1   | participant_age_group1   | participant_gender1   | min_age_participants   | avg_age_participants   | max_age_participants   | n_participants_child   | n_participants_teen   | n_participants_adult   | n_males   | n_females   | n_killed   | n_injured   | n_arrested   | n_unharmed   | n_participants   | notes          | incident_characteristics1   | incident_characteristics2   |
|:---------------|:-----------|:---------|:-----------------|:----------|:-----------|:------------|:-------------------------|:-----------------------|:------------------------|:-------------------|:-------------------------|:----------------------|:-----------------------|:-----------------------|:-----------------------|:-----------------------|:----------------------|:

## Columns filter
Select which columns to keep for the analysis
### Incidents
We keep `['date', 'state', 'city_or_county', 'latitude', 'longitude', 'congressional_district', 'n_participants_child', 'n_participants_teen', 'n_participants_adult', 'n_males', 'n_females', 'n_killed', 'n_injured', 'n_arrested', 'n_unharmed', 'n_participants', 'incident_characteristics1']`
### Poverty
We keep everything
### State District House
We keep everything

In [131]:
fincidents = incidents[['date', 'state', 'city_or_county', 'latitude', 'longitude', 'congressional_district', 'n_participants_child', 'n_participants_teen', 'n_participants_adult', 'n_males', 'n_females', 'n_killed', 'n_injured', 'n_arrested', 'n_unharmed', 'n_participants', 'incident_characteristics1']].copy()
print(f"All rows count: {fincidents.shape[0]}")
fincidents = fincidents[fincidents['n_participants'] != 0]
print(f"Zero participants removed: {fincidents.shape[0]}")
# fincidents[~fincidents.isna().any(axis=1)]
na_counts = fincidents.isna().sum(axis=1)

print(f"nan percentages:\n{na_counts.value_counts().sort_index() / fincidents.shape[0] * 100}")

fincidents[(na_counts > 6)]

All rows count: 239677
Zero participants removed: 214814
nan percentages:
0     84.764494
1      1.437057
2      2.443975
3      7.568874
4      0.114518
5      2.872718
6      0.188070
7      0.522312
8      0.085190
10     0.002793
Name: count, dtype: float64


,date,state,city_or_county,latitude,longitude,congressional_district,n_participants_child,n_participants_teen,n_participants_adult,n_males,n_females,n_killed,n_injured,n_arrested,n_unharmed,n_participants,incident_characteristics1
44,2014-10-25,MICHIGAN,Clarkston,42.7313,-83.3795,8.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1.0,Institution/Group/Business
172,2014-08-15,FLORIDA,Poinciana,28.1418,-81.4501,9.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,2.0,Armed robbery with injury/death and/or evidenc...
579,2015-04-20,ILLINOIS,Rockford,42.3014,-89.0912,17.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,2.0,Shots Fired - No Injuries
617,2014-05-21,ILLINOIS,Sterling,41.7922,-89.7334,17.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1.0,Armed robbery with injury/death and/or evidenc...
1394,2014-05-16,IOWA,Cedar Rapids,41.9841,-91.6276,1.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1.0,Armed robbery with injury/death and/or evidenc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238003,2015-03-08,SOUTH CAROLINA,Rock Hill,34.9169,-81.0107,5.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1.0,Officer Involved Incident
238871,2014-01-15,WISCONSIN,Milwaukee,43.0898,-87.9807,4.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1.0,Armed robbery with injury/death and/or evidenc...
239116,2014-01-06,NEW JERSEY,Bridgeton,39.4245,-75.2413,2.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1.0,Home Invasion
239261,2016-09-16,KENTUCKY,Louisville,38.2662,-85.8157,3.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,1.0,Non-Shooting Incident
